In [5]:
import psycopg2
import pandas as pd
import numpy as np
import constants as c
from dataclasses import dataclass
from typing import List
from tqdm import tqdm

@dataclass
class song:
    id : str
    name : str
    artists : List[str]
    danceability : float
    energy : float
    key : float
    loudness : float
    mode : float
    speechiness : float
    acousticness : float
    instrumentalness : float
    liveness : float
    valence : float
    tempo : float
    duration_ms : int
    time_signature : int
        
@dataclass
class playlist:
    name : str
    id : str
    tracks_url : str
    owner_id : str
    genres : List[str]
    songs : List[song]

class DatasetGenerator:
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.sql_conn('PlaylistScraper') 
    
    def sql_conn(self, db_name):
        self.conn = psycopg2.connect(host="localhost", port=5432, database=db_name, user=c.user, password=c.password)
        self.cur = self.conn.cursor()
        
        return
    
#   note this will only work with one col at a time
    def get_all_songs(self, col):
        try:
            q = \
            f'''
            SELECT 
            {col}
            FROM songs
            '''  
            
            self.cur.execute(q)
            data = self.cur.fetchall()
            
            return data
        
        except Exception as e:
            self.sql_conn('PlaylistScraper') 
            print(e)
            
            
            
        
    def get_songs(self, playlist_id):
        try:
            q = \
            """
            SELECT 
            s.id, s.name, s.artists_id,
            f.danceability, f.energy, f.key, f.loudness, f.mode, f.speechiness, f.acousticness, f.instrumentalness, f.liveness, f.valence, f.tempo, f.duration_ms, f.time_signature
            FROM playlist_songs p 
            INNER JOIN songs as s 
            ON p.song_id = s.id
            INNER JOIN song_features as f
            ON f.id = p.song_id
            where p.playlist_id = %s
            """
            self.cur.execute(q, (playlist_id,))
            raw_songs = self.cur.fetchall()
            songs = [song(*s) for s in raw_songs]

            return songs

        except Exception as e:
            self.sql_conn('PlaylistScraper') 
            print(e)
            
    def get_genres(self, playlist_id):
        try:
            q = "select genre from playlist_genres where playlist_id = %s"
            self.cur.execute(q, (playlist_id,))
            raw_genres = self.cur.fetchall()
            return [x[0] for x in raw_genres]
        
        except Exception as e:
            self.sql_conn('PlaylistScraper') 
            print(e)
    
    def get_playlist(self, playlist_id):
        try:
            q = "select * from playlists where id = %s"

            self.cur.execute(q, (playlist_id,))
            p = list(self.cur.fetchall()[0])

            genres = self.et_genres(playlist_id)
            p.append(genres)
            p.append(self.get_songs(playlist_id))

            return playlist(*p)

        except Exception as e:
            self.sql_conn('PlaylistScraper') 
            print(e)
            
    def get_playlists(self):
        q = "select id from playlists"
        self.cur.execute(q)
        return self.cur.fetchall()

    
    def generator(self, ids, batch_size):
        i = 0

        batches = ( len(ids) // batch_size )  + 1
        while i < batches:
            batch_ids = ids[i*batch_size : (i + 1) * batch_size]
            playlists = []
            for _id in batch_ids:
                playlists.append(get_playlist(_id))


            i += 1

            yield playlists
            
            
    def create_generator(self):
        _ids = self.get_playlists()
        return generator(_ids, self.batch_size)


In [6]:
dg = DatasetGenerator(100)
data = dg.get_all_songs('id')

In [8]:
dataset = set(data)

In [9]:
len(data)

9994149

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame(data)

In [13]:
df

,0
0,6ys9oyFvw7FXbs5UMZ7I7s
1,0HFx7PLqzGxSfN59j3UHmR
2,5gJKsGij5oGt5H5RSFYXPa
3,5D6W6RlRUdPYPSmkFVZ7xq
4,6RyWP3PCAXosigGRL3VbfW
...,...
9994144,2HypwKDzWtFGRh632qMUB8
9994145,4ZXxmmsp1Nd7ejbHs2BNBG
9994146,3QARHwKApSftZsT6mGSYX6
9994147,2SwkHQpyK3Tq11RtJjQhJY


In [61]:
def sql_conn(db_name):
    conn = psycopg2.connect(host="localhost", port=5432, database=db_name, user=c.user, password=c.password)
    cur = conn.cursor()

    return (conn, cur)

In [64]:
cols = ['id', 'name', 'artists_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
conn, cur  = sql_conn('PlaylistScraper')

In [65]:
q = \
"""
SELECT s.id, s.name, s.artists_id,
f.danceability, f.energy, f.key, f.loudness, f.mode, f.speechiness, f.acousticness, f.instrumentalness, f.liveness, f.valence, f.tempo, f.duration_ms, f.time_signature
FROM songs s
INNER JOIN song_features f
ON f.id = s.id
"""

# q = \
# """
# SELECT  p.playlist_id
# FROM playlist_songs p
# WHERE p.song_id = %s
# """


In [66]:
cur.execute(q, (_id,))
t = cur.fetchall()

('04V5A67zN9chglBmG4Qk38',
 'Walk On By (Saint Etienne Remix)',
 '["0bnwha8WPTU2sf4Q4P6vE0"]',
 0.771,
 0.474,
 9.0,
 -8.442,
 0.0,
 0.049,
 0.0274,
 0.0592,
 0.341,
 0.331,
 97.021,
 320073,
 4)

['id',
 'name',
 'artists_id',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature']

In [69]:
df = pd.DataFrame(t, columns=cols)

In [70]:
df

,id,name,artists_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,04V5A67zN9chglBmG4Qk38,Walk On By (Saint Etienne Remix),"[""0bnwha8WPTU2sf4Q4P6vE0""]",0.771,0.4740,9.0,-8.442,0.0,0.0490,0.027400,0.059200,0.3410,0.3310,97.021,320073,4
1,04vzdMVhahIOtzDRkjDB3y,Violence Condoned,"[""4jvgoG685KIVaPYwqVF60B""]",0.289,0.9960,7.0,-3.542,1.0,0.2360,0.000899,0.000014,0.2580,0.1070,120.461,142748,4
2,04WoNMS6KSW9oAD5eKCG3n,Diversion,"[""1e2pp6QqwCTfN5UnAdOhik"", ""63nvmtlUxf41sn03Xn...",0.285,0.0979,3.0,-21.903,0.0,0.0398,0.992000,0.869000,0.3350,0.0727,140.969,434000,4
3,04yNSjUdTO8xlHrVps3nAd,Zikir Ma'Thurat 11,"[""0ymK98HnydpVFP70dLb4LJ""]",0.762,0.0780,9.0,-11.774,1.0,0.0424,0.911000,0.000000,0.1370,0.8960,84.933,60333,4
4,04ZpWTZMV5ZYOK3Wy0qS9j,Icy (feat. Young Jeezy) - Club Mix,"[""13y7CgLHjMVRMDqxdx0Xdo"", ""4yBK75WVCQXej1p04G...",0.766,0.7030,10.0,-4.620,0.0,0.1060,0.051500,0.000000,0.1280,0.5970,92.092,296080,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993338,04oXtC6fkQOaZLfRS3pvCS,Flies And Spiders - Extended Version,"[""0OcclcP5o8VKH2TRqSY2A7""]",0.210,0.3010,7.0,-15.629,1.0,0.0444,0.569000,0.887000,0.3230,0.0383,120.439,575507,4
9993339,04Qf9iPchOds1OILUhSxdK,Cosmos,"[""4soOEEvChku18jdllY4IfG""]",0.600,0.5130,1.0,-12.147,1.0,0.0350,0.623000,0.935000,0.1070,0.6210,90.003,539247,4
9993340,04rDhSNzGMixiirYk2eZ8S,"Behind The Scenes (feat. Yukmouth, Poppa LQ, &...","[""4G1l1rhyOuN8wfAYfZWcOh"", ""0ndMofuAPmTktjONZv...",0.867,0.5940,2.0,-6.551,1.0,0.2470,0.011900,0.000000,0.0397,0.8170,91.967,295387,4
9993341,04sxYCSapTV33D3VpNqcQ0,X-Pollination - Mark Reeve Remix,"[""1u7DsNFbakULvxnDGtMm90"", ""58zZkWGb6qvx0AJ6L3...",0.831,0.8470,1.0,-6.509,1.0,0.0832,0.000821,0.833000,0.0941,0.4270,126.009,457143,4


In [71]:
import gc

In [78]:
del conn
del cur

In [79]:
gc.collect()

56

In [80]:
t


NameError: name 't' is not defined

In [81]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))

__name__ 57
__doc__ 113
__package__ 16
__loader__ 16
__spec__ 16
__builtin__ 72
__builtins__ 72
_ih 776
_oh 1176
_dh 64
In 776
Out 1176
get_ipython 64
exit 48
quit 48
_ 28
__ 28
___ 28
_i 58
_ii 61
_iii 65
_i1 258
sql_conn 136
_i2 89
_i3 252
_i4 89
_i5 3896
psycopg2 72
pd 72
np 72
c 72
dataclass 136
List 48
tqdm 1064
song 1064
playlist 1064
DatasetGenerator 1064
_i6 252
_i7 89
_i8 126
q 308
_i9 82
_i10 58
_10 64
_i11 122
_i12 82
_i13 58
_13 64
_i14 146
_i15 82
_i16 58
_16 64
_i17 147
_i18 82
_i19 89
_i20 147
_i21 82
_i22 58
_22 64
_i23 358
_i24 82
_i25 58
_25 64
_i26 539
_i27 82
_i28 542
_i29 82
_i30 553
_i31 82
_i32 89
_i33 553
_i34 82
_i35 58
_35 5224
_i36 526
_i37 82
_i38 58
_38 5224
_i39 55
_39 28
_i40 492
_i41 82
_i42 55
_42 28
_i43 120
_id 71
_i44 440
_i45 90
_i46 120
_i47 428
_i48 90
_i49 55
_49 28
_i50 58
_50 64
_i51 95
_51 69
_i52 237
_52 216
_i53 106
_53 152
_i54 247
_54 216
_i55 247
_55 216
_i56 248
_56 216
_i57 254
cols 184
_i58 53
_58 184
_i59 80


KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.lib.c_is_list_like'
Traceback (most recent call last):
  File "/home/moyez/anaconda3/envs/PyTorch/lib/python3.8/abc.py", line 96, in __instancecheck__
    def __instancecheck__(cls, instance):
KeyboardInterrupt: 


df 3522454857
_i60 51
_60 16
_i61 228
_i62 89
_i63 412
_i64 294
_i65 412
_i66 90
_i67 58
_67 79946800
_i68 53
_68 168
_i69 83
_i70 51
_70 3522454857
_i71 58
gc 72
_i72 54
_i73 61
_73 28
_i74 61
_74 28
_i75 61
_75 28
_i76 61
_76 28
_i77 58
_i78 65
_i79 61
_79 28
_i80 58
_i81 212
print_function 48
sys 72
